In [22]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
import itertools

In [23]:
def denoise_image(image, filters):
    '''
    filters=[
        {name:filter1
            n_iter:...,
            params:{
                ...
            }
        },
        {name:filter2
            n_iter:...,
            params:{
                ...
            }
        },
        ...
    ]
    '''
    for filter in filters:
        for _ in range(filter['n_iter']):
            match filter['name']:
                case "mean":
                    '''
                    {"ksize" : 15}
                    '''
                    k_size = filter['params']['ksize']
                    mean_kernel = np.ones([k_size, k_size])/(k_size**2)
                    image = cv2.filter2D(image, -1, mean_kernel)

                case "median":
                    '''
                    {"ksize" : 11}
                    '''
                    image = cv2.medianBlur(image, **filter['params'])

                case "bilateral":
                    '''
                    {"d" : 3,
                    "sigmaColor" : 2,
                    "sigmaSpace" : 1.5}
                    '''
                    image = cv2.bilateralFilter(
                        image, **filter['params'])

                case 'gaussian':
                    '''
                    {"sigmaX" : 2}
                    '''
                    sigmaX = filter['params']['sigmaX']
                    ksize = 2*int(np.ceil(3*sigmaX)) + 1
                    image = cv2.GaussianBlur(image, (ksize, ksize), sigmaX)

                case 'non_local_means':
                    '''
                    {'templateWindowSize':5,"searchWindowSize":21, "h":3, "hColor": 20}
                    '''
                    image = cv2.fastNlMeansDenoisingColored(
                        image, **filter['params'])

                case 'canny':
                    '''
                    {'threshold1':5,"threshold2":21}
                    '''
                    image = cv2.Canny(image, **filter['params'])

                # case "adaptive":
                #     channel1 = image[:, :, 0]
                #     channel2 = image[:, :, 1]
                #     channel3 = image[:, :, 2]
                #     image[:, :, 0] = adaptive_approach(channel1)
                #     image[:, :, 1] = adaptive_approach(channel2)
                #     image[:, :, 2] = adaptive_approach(channel3)

                case 'sharpen':
                    kernel = np.array([[0, -1, 0],
                                       [-1, 5, -1],
                                       [0, -1, 0]])
                    image = cv2.filter2D(image, -1, kernel=kernel)

                case 'emboss':
                    kernel = np.array([[-2, -1, 0],
                                       [-1, 1, 1],
                                       [0, 1, 2]])
                    image = cv2.filter2D(image, -1, kernel=kernel)
                case _: pass
    return image

In [42]:
''' 
USA SOLO I FILTRI NELLA LISTA E ITERA SOLO SU ORDINE E NUMERO DI ITERAZIONI + ALTRI PARAMETRI
'''
def generate_order_permutations(dict_list):
    order_permutations = []
    dict_count = len(dict_list)
    
    # Generate all permutations of the indices of the dictionaries
    for permutation in itertools.permutations(range(dict_count)):
        permuted_list = [dict_list[i] for i in permutation]
        order_permutations.append(permuted_list)
    
    return order_permutations

def calculate_total_iterations(permutazioni):
    total_iterations = 0
    for permutation in permutazioni:
        for filt in permutation:
            n_iter = len(filt['n_iter'])
            param_combinations_count = len(list(ParameterGrid(filt['params'])))
            total_iterations += n_iter * param_combinations_count
            print(f"In permutazione {filt}: {n_iter * param_combinations_count}")
    return total_iterations*len(permutazioni)


def find_best_params(noisy_image_path:str, 
                     denoised_image_path:str, 
                     filters:list
                     ) :
    # Leggo le immagini 
    input_image = cv2.imread(noisy_image_path)              
    input_image_denoised = cv2.imread(denoised_image_path)  
    
    # Per ogni possibile permutazione dei filtri richiesti
    permutazioni = generate_order_permutations(filters)
    
    # Calcola il numero totale di iterazioni
    total_iterations = calculate_total_iterations(permutazioni)#; print(total_iterations)

    # Prova tutte le possibili combinazioni di parametri
    best_score = float('inf')
    best_params = None
    with tqdm(total=total_iterations, desc="Testing Combinations") as pbar:
        for param_grid in permutazioni:
            print(param_grid)
            for params in ParameterGrid(param_grid):
                filtered_image = denoise_image(input_image, params)
                # Calcola una qualche metrica di qualità dell'immagine filtrata, ad esempio l'errore quadratico medio
                score = np.mean((input_image_denoised - filtered_image) ** 2)
                # Aggiorna i parametri se otteniamo una migliore qualità dell'immagine filtrata
                if score < best_score:
                    best_score = score
                    best_params = params
                # Aggiorna la barra di avanzamento
                pbar.update(1)

    print("Migliori parametri:", best_params)

    # Applica il bilateral filtering con i migliori parametri
    filtered_image = denoise_image(input_image, best_params)

    # Visualizza l'immagine filtrata
    plt.figure(figsize=(20,20))
    plt.subplot(131); plt.imshow(input_image[:,:,::-1]); plt.title("Input Image")
    plt.subplot(132); plt.imshow(input_image_denoised[:,:,::-1]); plt.title("Perfectly-denoised Image")
    plt.subplot(133); plt.imshow(filtered_image[:,:,::-1]); plt.title("Filtered Image")
    plt.show()

    # Visualizza errore
    plt.figure(figsize=(10,10))
    plt.imshow(abs(input_image[:,:,::-1]-filtered_image[:,:,::-1])); plt.title("Filtering error")
    
    return best_params

In [44]:
path_noised = "./Users/pelle/Desktop/scene4.png"
path_DEnoised = "./Users/pelle/Desktop/scene4_denoised_online.png"

best_params = find_best_params(
    noisy_image_path=path_noised,
    denoised_image_path=path_DEnoised,
    filters=[
        {   
            "name": ["median"],
            "n_iter": [7],  # Modifica qui, avvolgi il valore in una lista
            "params": {
                "ksize": [*range(3, 11, 2)]
            }
        },
        {   
            "name": ["median"],
            "n_iter": [7],  # Modifica qui, avvolgi il valore in una lista
            "params": {
                "ksize": [*range(3, 11, 2)]
            }
        },
    ]
)


In permutazione {'name': ['median'], 'n_iter': [7], 'params': {'ksize': [3, 5, 7, 9]}}: 4
In permutazione {'name': ['median'], 'n_iter': [7], 'params': {'ksize': [3, 5, 7, 9]}}: 4
In permutazione {'name': ['median'], 'n_iter': [7], 'params': {'ksize': [3, 5, 7, 9]}}: 4
In permutazione {'name': ['median'], 'n_iter': [7], 'params': {'ksize': [3, 5, 7, 9]}}: 4


Testing Combinations:   0%|          | 0/32 [00:00<?, ?it/s]

[{'name': ['median'], 'n_iter': [7], 'params': {'ksize': [3, 5, 7, 9]}}, {'name': ['median'], 'n_iter': [7], 'params': {'ksize': [3, 5, 7, 9]}}]


TypeError: Parameter grid for parameter 'params' needs to be a list or a numpy array, but got {'ksize': [3, 5, 7, 9]} (of type dict) instead. Single values need to be wrapped in a list with one element.